# Imports

In [12]:
# !pip install scikit-learn
# !pip install xgboost
# !pip install tqdm
#

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)


In [8]:
# Dependencies: numpy, scipy, scikit-image, opencv-python, pywt, sklearn, torchvision, torch
import numpy as np
import cv2
from skimage.feature import greycomatrix, greycoprops, local_binary_pattern
from skimage.filters import gabor
from skimage.filters import laplace
import pywt
from scipy.stats import skew, kurtosis, entropy as shannon_entropy
from torchvision import models, transforms
import torch
from glob import glob
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import xgboost as xgb
from skimage.feature import graycomatrix, graycoprops
from torchvision.models import ResNet50_Weights
from joblib import Parallel, delayed
from tqdm import tqdm
import pandas as pd

# Defenition

In [2]:

def basic_intensity_features(img):
    p = np.percentile(img, [10,25,50,75,90])
    hist = np.histogram(img, bins=256, range=(0,255))[0] + 1  # add 1 to avoid log(0)
    return {
        'mean': img.mean(),
        'std': img.std(),
        'skew': skew(img.ravel()),
        'kurt': kurtosis(img.ravel()),
        'p10': p[0], 'p25': p[1], 'p50': p[2], 'p75': p[3], 'p90': p[4],
        'entropy': shannon_entropy(hist)
    }

def glcm_features(img, distances=[1,2,4], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4]):
    # convert to uint8 levels 0..255 then to e.g. 32 levels to reduce GLCM size
    levels = 32
    im = (img / (256/levels)).astype(np.uint8)
    glcm = graycomatrix(im, distances=distances, angles=angles, levels=levels, symmetric=True, normed=True)
    props = ['contrast','dissimilarity','homogeneity','energy','correlation','ASM']
    feats = {}
    # glcm = graycomatrix(img, distances=[1], angles=[0], symmetric=True, normed=True)

    for p in props:
        # arr = greycoprops(glcm, p)
        arr = graycoprops(glcm, p)

        feats[p+'_mean'] = arr.mean()
        feats[p+'_std'] = arr.std()
    return feats

def lbp_hist(img, P=8, R=1, bins=16):
    im = img.astype(np.uint8)
    lbp = local_binary_pattern(im, P, R, method='uniform')
    (hist,_) = np.histogram(lbp.ravel(), bins=bins, range=(0, bins))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)
    return {f'LBP_{i}': hist[i] for i in range(bins)}

def gabor_features(img, frequencies=[0.1,0.2], thetas=[0, np.pi/4, np.pi/2, 3*np.pi/4]):
    feats = {}
    for freq in frequencies:
        for theta in thetas:
            real, imag = gabor(img, frequency=freq, theta=theta)
            mag = np.sqrt(real**2 + imag**2)
            feats[f'gabor_f{freq}_t{theta}_mean'] = mag.mean()
            feats[f'gabor_f{freq}_t{theta}_std'] = mag.std()
    return feats

def wavelet_energy(img, wavelet='db2', level=2):
    coeffs = pywt.wavedec2(img.astype(float), wavelet=wavelet, level=level)
    energies = {}
    for i, arr in enumerate(coeffs[1:], start=1):
        # arr is tuple (cH, cV, cD)
        cH, cV, cD = arr
        energies[f'wl_L{i}_cH'] = np.sum(cH**2)
        energies[f'wl_L{i}_cV'] = np.sum(cV**2)
        energies[f'wl_L{i}_cD'] = np.sum(cD**2)
    return energies

# Deep features (ResNet50 global pool)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# resnet = models.resnet50(pretrained=True)
resnet = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)

resnet = torch.nn.Sequential(*list(resnet.children())[:-1]).to(device).eval()  # remove last fc
preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

def extract_deep(img):  # img: HxW uint8 gray or RGB
    if img.ndim==2:
        img = np.stack([img]*3, axis=-1)
    x = preprocess(img).unsqueeze(0).to(device)
    with torch.no_grad():
        feat = resnet(x).cpu().numpy().reshape(-1)
    return feat  # 2048-d


In [3]:
from skimage.filters import laplace, gaussian
from skimage.filters.rank import entropy
from skimage.morphology import disk
from skimage.feature import canny

def extract_all_features_extended(img, use_deep=True):
    """
    Extract all features (handcrafted + optional deep) from a single RGC image.
    """
    features = {}

    # --- 1. Basic intensity features ---
    features.update(basic_intensity_features(img))

    # --- 2. GLCM features ---
    try:
        features.update(glcm_features(img))
    except Exception as e:
        print("Warning: GLCM skipped:", e)

    # --- 3. LBP histogram ---
    features.update(lbp_hist(img))

    # --- 4. Gabor features ---
    features.update(gabor_features(img))

    # --- 5. Wavelet energies ---
    features.update(wavelet_energy(img))

    # --- 6. Edge density ---
    edges = canny(img.astype(np.uint8))
    features['edge_density'] = edges.mean()

    # --- 7. LoG stats ---
    log_img = laplace(gaussian(img, sigma=1))
    features['LoG_mean'] = log_img.mean()
    features['LoG_std'] = log_img.std()

    # --- 8. Local entropy ---
    try:
        entropy_img = entropy(img.astype(np.uint8), disk(5))
        features['local_entropy_mean'] = entropy_img.mean()
        features['local_entropy_std'] = entropy_img.std()
    except Exception as e:
        print("Warning: Local entropy skipped:", e)

    # --- 9. Deep features ---
    if use_deep:
        try:
            deep_feat = extract_deep(img)
            features.update({f'deep_{i}': deep_feat[i] for i in range(len(deep_feat))})
        except Exception as e:
            print("Warning: Deep features skipped:", e)

    return features


## Test the feature extraction with one sample

In [5]:

img_add="./Dataset/train/dic/1ld1.png"
img = cv2.imread(img_add, cv2.IMREAD_GRAYSCALE)
img = img.astype(np.float64)

feats = extract_all_features_extended(img, use_deep=True)  # False if you skip PyTorch

# Convert to DataFrame for RF/XGBoost
import pandas as pd
df = pd.DataFrame([feats])


In [6]:
df

,mean,std,skew,kurt,p10,p25,p50,p75,p90,entropy,...,deep_2038,deep_2039,deep_2040,deep_2041,deep_2042,deep_2043,deep_2044,deep_2045,deep_2046,deep_2047
0,155.208651,12.350875,1.136561,2.078124,141.0,146.0,153.0,162.0,171.0,3.790159,...,0.006279,0.010693,0.00583,0.246146,0.364183,0.144402,0.196051,0.289868,0.0,1.297428


# Create the Dataset feature maps

## Extract features - it takes too long :(


In [ ]:

train_dic="./Dataset/train/dic/*.png"
image_files = glob(train_dic)  # or .jpg
all_features = []

for file in image_files:
    print(file)
    img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    img = img.astype(np.float64)

    feats = extract_all_features_extended(img, use_deep=True)  # or True

    feats['filename'] = file
    all_features.append(feats)

# Convert to DataFrame
df = pd.DataFrame(all_features)
df.fillna(0, inplace=True)  # replace any missing values with 0

# Save to CSV
df.to_csv("./Feature_maps/train_rgc_features_f64.csv", index=False)

## Extract features using concurrent workers

In [ ]:
def process_image(file):
    """Process one image and extract features."""
    try:
        img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
        img = img.astype(np.float64)

        feats = extract_all_features_extended(img, use_deep=True)
        feats['filename'] = file
        return feats
    except Exception as e:
        print(f"Error processing {file}: {e}")
        return None


# image folder
sets= ['train' , 'test' , 'eval']
for set in sets:
    folder_dic = "./Dataset/"+ set +"/dic/*.png"
    image_files = glob(folder_dic)
    # Use tqdm to wrap the iterable
    results = Parallel(n_jobs=8, backend='threading')(
        delayed(process_image)(file) for file in tqdm(image_files, desc="Extracting features")
    )

    # Filter out None results
    all_features = [f for f in results if f is not None]

    # Convert to DataFrame and save
    df = pd.DataFrame(all_features)
    df.fillna(0, inplace=True)
    df.to_csv("./Feature_maps/"+ set +"_rgc_features_f64.csv", index=False)

    print("✅ Feature extraction complete. Saved to "+ set +"_rgc_features_f64.csv")


# Create the label Column based on the manual cell counts:

In [3]:
 # other ways of labeling the dataset
 # you can edit the margin of splitting the classes
 # 400–500 cells ==> central
 # 350–430 cells ==> middle
 # 250–300 cells ==> periphery
 #
# # Define expected ranges and their midpoints
# region_midpoints = {
#     "central":   np.mean([400, 500]),   # 450
#     "middle":    np.mean([350, 430]),   # 390
#     "periphery": np.mean([250, 300])    # 275
# }
#
# def assign_region(auto_count):
#     diffs = {region: abs(auto_count - mid) for region, mid in region_midpoints.items()}
#     return min(diffs, key=diffs.get)


# def assign_region(auto_count):
#     if 400 <= auto_count :
#         return "central"
#     elif 280 < auto_count < 400:
#         return "middle"
#     elif  auto_count <= 280:
#         return "periphery"
#     else:
#         return "unknown"


In [26]:
def assign_region(auto_count):
    if  auto_count <=300 :
        return "low"
    else:
        return "moderate"


In [27]:
labels_df = pd.read_csv("./Dataset/manaull_counts.csv")  # make sure path is correct
labels_df["region"] = labels_df["manual count"].apply(assign_region)
labels_df.rename(columns={"file name": "filename"}, inplace=True)

In [28]:
dataset=['train' , 'test' , 'eval']
for i in dataset:
    features_df =pd.read_csv('./Feature_maps/'+i+'_rgc_features_f64.csv') # after extraction loop
    features_df.fillna(0, inplace=True)
    features_df['filename'] = features_df['filename'].str.split("\\").str[-1]
    features_df['filename'] = features_df['filename'].str.split(".").str[0]
    final_df = pd.merge(features_df, labels_df, on="filename", how="inner")
    final_df.to_csv("./Feature_maps/"+ i+"_features_with_labels_f64_with_lables.csv", index=False)



# Start Training

# -------------------------------
# 1. Load data
# -------------------------------

In [ ]:

train_df = pd.read_csv("./train_features_with_labels_f64_with_lables.csv")
test_df  = pd.read_csv("./test_features_with_labels_f64_with_lables.csv")

# Drop filename column (not useful for training)
X_train = train_df.drop(columns=["filename", "region" , "manual count",	"auto count"])
y_train = train_df["region"]

X_test  = test_df.drop(columns=["filename", "region" , "manual count",	"auto count"])
y_test  = test_df["region"]
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
print(X_train.head())
print(y_train.head())

In [31]:

# Replace inf and -inf with NaN
X_train = X_train.replace([np.inf, -np.inf], np.nan)
X_test  = X_test.replace([np.inf, -np.inf], np.nan)

# Fill NaNs with column means (or 0 if you prefer)
X_train = X_train.fillna(X_train.mean())
X_test  = X_test.fillna(X_train.mean())  # use train mean for consistency

# Optional: convert to float32 if very large numbers exist
X_train = X_train.astype(np.float32)
X_test  = X_test.astype(np.float32)



# -------------------------------
# 2. Random Forest baseline
# -------------------------------

In [23]:

print("🔹 Training Random Forest...")
rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)

print("\nRandom Forest Results:")
print(confusion_matrix(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf, digits=3))

🔹 Training Random Forest...

Random Forest Results:
[[ 41   7]
 [  5 103]]
              precision    recall  f1-score   support

         low      0.891     0.854     0.872        48
    moderate      0.936     0.954     0.945       108

    accuracy                          0.923       156
   macro avg      0.914     0.904     0.909       156
weighted avg      0.922     0.923     0.923       156




# -------------------------------
# 3. XGBoost (usually stronger)
# -------------------------------

In [32]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)  # e.g. central->0, middle->1, periphery->2
y_test_enc  = le.transform(y_test)       # use same encoder


In [33]:

print("\n🔹 Training XGBoost...")
xgb_clf = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)
xgb_clf.fit(X_train, y_train_enc)

y_pred_enc = xgb_clf.predict(X_test)

# Convert back to original labels if needed
y_pred = le.inverse_transform(y_pred_enc)

print("\nXGBoost Re       sults:")
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=3))



🔹 Training XGBoost...

XGBoost Re       sults:
[[53  4]
 [ 3 96]]
              precision    recall  f1-score   support

         low      0.946     0.930     0.938        57
    moderate      0.960     0.970     0.965        99

    accuracy                          0.955       156
   macro avg      0.953     0.950     0.951       156
weighted avg      0.955     0.955     0.955       156



In [34]:

# Save the model
xgb_clf.save_model("./checkpoints/xgb_regions_detection_model.json")  # or .bin

# Use the saved model

In [ ]:

# Later, load the model
loaded_model = xgb.XGBClassifier()
loaded_model.load_model("xgb_regions_detection_model.json")
print(y_test[:5])

# Predict with it
y_pred = loaded_model.predict(X_test)
print(y_pred[:5])

y_pred = le.inverse_transform(y_pred)
print(y_pred[:5])